In [6]:
# importing tools 
import s3fs
import xarray as xr
import numpy as np 
import matplotlib.pyplot as plt 
import math
#for removing warnings
import warnings

In this scipt the data is NOT smoothened 

In [8]:
def open_s3fs(path):
    # open the data with s3fs from the databucket 
    fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
    data_path = f's3://data/{path}/*'
    remote_files = fsg.glob(data_path)
    fileset = [fsg.open(file) for file in remote_files]
    #open the dataset 
    dset = xr.open_mfdataset(fileset, combine='by_coords',compat='override')
    # rename variabel
    dset = dset.rename({'__xarray_dataarray_variable__' : 'vel'})
    return dset

In [9]:
dset = open_s3fs('PC-data')
dset

<xarray.Dataset>
Dimensions:        (depth: 75, x: 1580, y: 1801)
Coordinates:
  * depth          (depth) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    time_centered  object ...
    time_counter   object ...
    nav_lat        (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
Dimensions without coordinates: x, y
Data variables:
    vel            (depth, y, x) float32 dask.array<chunksize=(75, 1801, 1580), meta=np.ndarray>
    mbathy         (y, x) int16 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    vomecrty       (depth, y, x) float32 dask.array<chunksize=(75, 1801, 1580), meta=np.ndarray>
    vozocrtx       (depth, y, x) float32 dask.array<chunksize=(75, 1801, 1580), meta=np.ndarray>

In [10]:
# rewriting the name of the variables 
uvar = dset.vozocrtx.isel(y=slice(600,1800), x=slice(200,1400))
vvar = dset.vomecrty.isel(y=slice(600,1800), x=slice(200,1400))
uv = dset.vel.isel(y=slice(600,1800), x=slice(200,1400))

In [11]:
# Getting the bottom velocity 
d = dset.mbathy.isel(y=slice(600,1800), x=slice(200,1400)).values-1 
d[d==-1] = 0

d = xr.DataArray(d, dims=['y','x'])

uvar_b = uvar.isel(depth=d)
vvar_b = vvar.isel(depth=d)
uv_b = uv.isel(depth=d)

In [13]:
# List of x,y elements
dx = 20
dy = 20

jmax, imax = uv_b.shape
i_list = np.arange(0,imax,dx)
j_list = np.arange(0,jmax,dy)

In [14]:
# Making the covmatrix and finding the eigenvalues and eigenvectors
# ----------------------------------------------------------------------
PC11 = []
PC12 = []
PC21 = []
PC22 = []
y = []
x = []
indx = 0

In [ ]:
# to get the loop to run faster
uvy = np.array(uv.y.values)
uvx = np.array(uv.x.values)

# surface values
uvarb = np.array(uvar_b.values)
vvarb = np.array(vvar_b.values)
uvvel = np.array(uv_b.values)

for i in i_list:
    for j in j_list:
        covm = np.array([[uvar_depth[j,i],uv_depth[j,i]],
        [uv_depth[j,i],vvar_depth[j,i]]],dtype=float)
        if np.isnan(covm).any():
            continue
        else:
            values, vectors = np.linalg.eig(covm)
            PC11.append(vectors[0,0]*np.sqrt(values[0]))
            PC12.append(vectors[1,0]*np.sqrt(values[0]))
            PC21.append(vectors[0,1]*np.sqrt(values[1]))
            PC22.append(vectors[1,1]*np.sqrt(values[1]))
            y.append(uvy[j])
            x.append(uvx[i])
            indx += 1
# converting to np.arrays
PC11 = np.array(PC11,dtype=float)
PC12 = np.array(PC12,dtype=float)
PC21 = np.array(PC21,dtype=float)
PC22 = np.array(PC22,dtype=float)
y = np.array(y,dtype=float)
x = np.array(x,dtype=float)


In [ ]:
# Plot
fig, ax = plt.subplots(1,1,figsize=(15,15))
plt.title(f'PC of the current at ' + str(depth_list[n]) + 'm depth')
cmap = plt.get_cmap('magma_r')
ax.contour(dset.mbathy[600:1800,200:1400], colors='grey',levels=8, linewidths=0.4)
ax.quiver( x[::1], y[::1], PC11_s[::1], PC12_s[::1], 
    #color='r',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=2)
ax.quiver(x[::1], y[::1], -PC11_s[::1], -PC12_s[::1],
    #color='r',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=2)
ax.quiver( x[::1], y[::1], PC21_s[::1], PC22_s[::1],
    #color='b',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=2)
ax.quiver( x[::1], y[::1], -PC21_s[::1], -PC22_s[::1],
    #color='b',
    headwidth=1,
    headlength=1,
    width=0.002,
    scale=2)


plt.show()
plt.title('PC plot of the bottom current')

plt.show()